In [80]:
from z3 import *
from itertools import combinations, permutations, product

In [81]:
board1=[[9,0,3,4,2],2,[7,0,7,9,4],0,[3,9,4,5,8],2,[3,4,1,0,9],1,[5,1,5,4,5],2,[1,2,5,3,1],1]

board12=[[9,0,3,4,2],2,2,
         [7,0,7,9,4],0,2,
         [3,9,4,5,8],2,2,
         [3,4,1,0,9],1,2,
         [5,1,5,4,5],2,0,
         [1,2,5,3,1],1,2]

board=[[5,6,1,6,1,8,5,6,5,0,5,1,8,2,9,3],2,[3,8,4,7,4,3,9,6,4,7,2,9,3,0,4,7],1,[5,8,5,5,4,6,2,9,4,0,8,1,0,5,8,7],3,[9,7,4,2,8,5,5,5,0,7,0,6,8,3,5,3],3,[4,2,9,6,8,4,9,6,4,3,6,0,7,5,4,3],3,[3,1,7,4,2,4,8,4,3,9,4,6,5,8,5,8],1,[4,5,1,3,5,5,9,0,9,4,1,4,6,1,1,7],2,[7,8,9,0,9,7,1,5,4,8,9,0,8,0,6,7],3,[8,1,5,7,3,5,6,3,4,4,1,1,8,4,8,3],1,[2,6,1,5,2,5,0,7,4,4,3,8,6,8,9,9],2,[8,6,9,0,0,9,5,8,5,1,5,2,6,2,5,4],3,[6,3,7,5,7,1,1,9,1,5,0,7,7,0,5,0],1,[6,9,1,3,8,5,9,1,7,3,1,2,1,3,6,0],1,[6,4,4,2,8,8,9,0,5,5,0,4,2,7,6,8],2,[2,3,2,1,3,8,6,1,0,4,3,0,3,8,4,5],0,[2,3,2,6,5,0,9,4,7,1,2,7,1,4,4,8],2,[5,2,5,1,5,8,3,3,7,9,6,4,4,3,2,2],2,[1,7,4,8,2,7,0,4,7,6,7,5,8,2,7,6],3,[4,8,9,5,7,2,2,6,5,2,1,9,0,3,0,6],1,[3,0,4,1,6,3,1,1,1,7,2,2,4,6,3,5],3,[1,8,4,1,2,3,6,4,5,4,3,2,4,5,8,9],3,[2,6,5,9,8,6,2,6,3,7,3,1,6,8,6,7],2]

board2 = [
[1,1,2,2],0,1,
[3,4,4,1],1,1,
[5,3,5,1],1,1,
[6,6,3,1],0,1,
[2,4,5,3],1,3
]

In [82]:
def mastermind1_aux(B,P,s):

    # One digit must be true per position:
    for pos in range(len(P)):
        s.add(Or(P[pos]))
        # No more than one digit can be true per position:
        for digit in range(10):
            for i in range(digit+1,10):
                s.add(Not(And(P[pos][digit],P[pos][i])))
    for i in range(0,len(B),2):
        b = [] # List of relevant booleans for the current line of the board
        aux = []
        for j in range(len(B[i])):
            b.append(P[j][B[i][j]])
        if B[i+1] == 0:
            s.add(Not(Or(b))) # These variables must necessarily be false since no position was correct
        else:
            comb = list(combinations(b,B[i+1])) # List of tuples of length k, where k is the number of correct positions
            for tuples in comb:
                other = [ x for x in b if x not in tuples ] # Complement set of current tuple
                aux.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
            s.add(Or(aux)) # One of the previous conditions must hold from the set of all tuples
    if s.check() == sat:
        m = s.model()
        solution = []
        # Search boolean matrix for trues to present solution:
        for pos in range(len(P)):
            for digit in range(10):
                if m[P[pos][digit]]:
                    solution.append(digit)
        return solution
        
    else:
        print('No solutions found')

def mastermind1(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    return mastermind1_aux(B,P,s)
def unique_solution1(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    solution = mastermind1_aux(B,P,s)
    # Deny the solution given by mastermind1 in the model to see if it's unique:    
    s.add(Not(And([P[pos][solution[pos]] for pos in range(len(P))]))) # negating the solution (i.e. conjunction of the propositions)
    return s.check() == unsat and len(solution) > 0



def mastermind2_aux(B,P,s):
    
    # One digit must be true per position:
    for pos in range(len(P)):
        s.add(Or(P[pos]))
    # No more than one digit can be true per position:
        for digit in range(10):
            for i in range(digit+1,10):
                s.add(Not(And(P[pos][digit],P[pos][i])))

    for i in range(0,len(B),3):

        comb_black = list(combinations(list(enumerate(B[i])),B[i + 1])) # List of tuples of length k, where k is the number of correct positions

        aux = [] 
        posit = [x for x in range(len(B[i]))] # if this line doesn't exist, range(len(B[i])) will be called many times in loops
       
        for b in comb_black: # for every combination of possible correct positions
            bval = [x[1] for x in b] # list of possible combinations of correctly positioned digits
            bpos = [x[0] for x in b] # corresponding positions
            W_avlbl = [x for x in list(enumerate(B[i])) if x not in b] # remove the current "black combination" from the attempt
            comb_white = list(combinations(W_avlbl,B[i + 2])) # list of "white combinations", i.e. combinations of possible correct digits in wrong positions
            
            aux1 = []
            for w in comb_white: # for every combination of possible correct digits in incorrect positions
                wval = [x[1] for x in w] # list of "white" digit combinations, without their respective positions
                pos_avlbl = [x for x in posit if x not in bpos] # list of available positions for the "white" digits
                pos_for_w = list(permutations(pos_avlbl,B[i + 2])) # initial positions and permutations of possible positions (the initial positions weren't excluded yet)
                w_pssbl = [] # list of possible combinations of the "white" digits inserted in possible positions
                for x in pos_for_w:
                    w_comb = [] # combination of "white" digits inserted in possible positions 
                    for k in range(len(w)):
                        w_comb.append((x[k],w[k][1]))
                    w_pssbl.append(w_comb)

                other = [x for x in w] # complementary set, we start by excluding the possiblity of "white" digits being correct in their initial positions
                other_n = [x for x in B[i] if x not in bval + wval] # all the digits that are not in the code, if the current "black" and "white" digits are
                for p in posit: #these digits are incorrect in every position
                    for n in other_n:
                        other.append((p,n))
                
                for guess in w_pssbl:
                    aux1.append(And(And([P[x[0]][x[1]] for x in guess]), Not(Or([P[x[0]][x[1]] for x in other]))))
                    #aux1.append(And([P[x[0]][x[1]] for x in guess]))
            aux.append(And(And([P[x[0]][x[1]] for x in b]), Or(aux1)))

        s.add(Or(aux))
    solution = []
    if s.check() == sat:
        m = s.model()
        # Search boolean matrix for trues to present solution:
        for pos in range(len(P)):
            for digit in range(10):
                if m[P[pos][digit]]:
                    solution.append(digit)
    else:
        print('No solutions found')
    
    return solution

def mastermind2(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    solution = mastermind2_aux(B,P,s)
    return solution 

def unique_solution2(B):
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    solution = mastermind2_aux(B,P,s)
    s.add(Not(And([P[pos][solution[pos]] for pos in range(len(P))]))) # negating the solution (i.e. conjunction of the propositions)
    return s.check() == unsat and len(solution) > 0



In [ ]:
#find all solutions to board2

P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(board2[0])+1)]
s = Solver()
solution = mastermind2_aux(board2,P,s)
count = 0
found = False
print(s.check())
while s.check() == sat:
    m = s.model()
    solution = []
    # Search boolean matrix f   print(adds)or trues to present solution:
    for pos in range(len(P)):
        for digit in range(10):
            if m[P[pos][digit]]:
                solution.append(digit)
    count += 1
    print(f'Solution {count}: {solution}')
    found = True
    # Deny this solution in the model to search for others:
    s.add(Not(And([P[pos][solution[pos]] for pos in range(len(P))])))

sat
Solution 1: [3, 2, 5, 4]
Solution 2: [2, 3, 4, 5]
Solution 3: [5, 2, 4, 3]


In [84]:
mastermind1(board1)

[3, 9, 5, 4, 2]

In [85]:
unique_solution1(board1)

True

In [86]:
mastermind1(board)

[4, 6, 4, 0, 2, 6, 1, 5, 7, 1, 8, 4, 9, 5, 3, 3]

In [87]:
unique_solution1(board)

True

In [88]:
mastermind2(board12)

[3, 9, 5, 4, 2]

In [89]:
unique_solution2(board12)

True

In [90]:
mastermind2(board2)

[3, 2, 5, 4]

In [91]:
unique_solution2(board2)

False